In [3]:
#Initial commands

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

C:\Users\Will\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Before next step, make sure lines in CSV file are randomly shuffled.  Use code from http://stackoverflow.com/questions/4618298/randomly-mix-lines-of-3-million-line-file.  But make sure to keep the first line intact so you don't forget the column names!

Code needs to modified in the following way: <br>
data = lines[1:] <br>
newlines = [lines[0]] + data <br>
write(newlines)

In [4]:
#Import data in batches instead of all at once

OnTimeDataFrames = []
for n in range(15):
    #100,001 rows total: 100,000 of data, plus first row for column names!  Easy to mess up...
    #Note we aren't using lat/long data in the model at the present time, but can change this later
    OnTimeDataFrames.append(pd.read_csv('Sample_2011_2016_carr_mark.csv', nrows=100001,
                                        skiprows=range(1, 1 + 100000*n) if n > 0 else None))

In [5]:
#Check shapes of all data frames - all except the last should have 100,000 rows, all should have same no. of columns
for X in OnTimeDataFrames: print X.shape

(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(100001, 68)
(90434, 68)


In [6]:
#Check column names of second data frame
OnTimeDataFrames[1].columns

Index([u'Unnamed: 0', u'Year', u'Quarter', u'Month', u'DayofMonth',
       u'DayOfWeek', u'FlightDate', u'UniqueCarrier', u'AirlineID', u'Carrier',
       u'TailNum', u'FlightNum', u'OriginAirportID', u'OriginAirportSeqID',
       u'OriginCityMarketID', u'Origin', u'OriginCityName', u'OriginState',
       u'OriginStateFips', u'OriginStateName', u'OriginWac', u'DestAirportID',
       u'DestAirportSeqID', u'DestCityMarketID', u'Dest', u'DestCityName',
       u'DestState', u'DestStateFips', u'DestStateName', u'DestWac',
       u'CRSDepTime', u'DepTime', u'DepDelay', u'DepDelayMinutes', u'DepDel15',
       u'DepartureDelayGroups', u'DepTimeBlk', u'TaxiOut', u'WheelsOff',
       u'WheelsOn', u'TaxiIn', u'CRSArrTime', u'ArrTime', u'ArrDelay',
       u'ArrDelayMinutes', u'ArrDel15', u'ArrivalDelayGroups', u'ArrTimeBlk',
       u'Cancelled', u'CancellationCode', u'CRSElapsedTime',
       u'ActualElapsedTime', u'AirTime', u'Flights', u'Distance',
       u'DistanceGroup', u'CarrierDelay', u'Weat

In [7]:
#Check that all data frames have same column names
assert np.all(X.columns == OnTimeDataFrames[0].columns for X in OnTimeDataFrames)

In [8]:
#Correct ExpressJet Airways issue
for X in OnTimeDataFrames:
    X["Carrier Name"] = X["Carrier Name"].replace("ExpressJet Airlines Inc. (1)",
                                                               "ExpressJet Airlines Inc.")

In [9]:
#Get rid of rows with null values for ArrDelay
OnTimeDataNoNull = [X[np.isfinite(X.ArrDelay)] for X in OnTimeDataFrames]
for X in OnTimeDataNoNull: print X.shape

(98231, 68)
(98163, 68)
(98252, 68)
(98174, 68)
(98197, 68)
(98184, 68)
(98200, 68)
(98205, 68)
(98198, 68)
(98190, 68)
(98234, 68)
(98315, 68)
(98177, 68)
(98145, 68)
(88801, 68)


In [10]:
#Compute interactions for airlines * airports with a certain %
def interactions_aa(X, pct):
    #dictionary of features with percentage "1"s
    PercentageDict = {}
    for feature in X.columns:
        PercentageDict[feature] = X[feature].mean(axis=1)
    #calculate interactions
    X1 = X.copy()
    for feat_A in [name for name in X.columns if name[0] == 'C']:
        for feat_B in [name for name in X.columns if name[0] in list('DO') and PercentageDict[name] > pct]:
            X1[feat_A + "*" + feat_B] = (X1[feat_A].to_dense() * X1[feat_B].to_dense()).to_sparse()
    return X1

In [11]:
#Create dummy variables
def DummyRegressors(data):
    #Create dummies for airline, origin and destination airports, month, and time of day
    #Drop first column in each data frame to avoid singular matrix
    #Also include ArrDelay variable so we don't have to grab this separately from the old data frame
    CarrierDummies = pd.get_dummies(data["Carrier Name"], prefix="C", sparse=True, drop_first=True)
    OriginDummies = pd.get_dummies(data.Origin, prefix="O", sparse=True, drop_first=True)
    DestDummies = pd.get_dummies(data.Dest, prefix="D", sparse=True, drop_first=True)
    MonthDummies = pd.get_dummies(data.Month, prefix="M", sparse=True, drop_first=True)
    TimeDummies = pd.get_dummies(data.DepTimeBlk, prefix="T", sparse=True, drop_first=True)
    ArrDelay = data.ArrDelay
    
    return pd.concat([CarrierDummies, OriginDummies, DestDummies, MonthDummies, TimeDummies, ArrDelay], axis=1)

In [12]:
#Create list of data frames with dummies instead of raw data
OnTimeDummies = [DummyRegressors(X) for X in OnTimeDataNoNull]

In [13]:
for X in OnTimeDummies: print X.shape

(98231, 701)
(98163, 705)
(98252, 702)
(98174, 702)
(98197, 704)
(98184, 704)
(98200, 695)
(98205, 706)
(98198, 706)
(98190, 703)
(98234, 698)
(98315, 697)
(98177, 706)
(98145, 707)
(88801, 700)


Problem: not all airports and airlines are in all the data frames!  This makes sense and is a drawback of reading in the data in separate frames rather than splitting it up afterward.  We need to fix it by making sure if a column name appears in one dummy frame, it appears in all of them.

In [14]:
#create set of dummies that appear in any data frame, so each only appears once
#Try it the less efficient way, but this seems to work
DummyList = []
for X in OnTimeDummies:
    for name in X.columns:
        DummyList.append(name)
DummySet = set(DummyList)
len(DummySet)

717

In [15]:
len(DummySet)

717

In [16]:
#Now add dummies for all variables in set to OnTimeDummies frames
for X in OnTimeDummies:
    SparseZeroes = pd.SparseSeries([0 for n in range(len(X))])
    for name in DummySet:
        if name not in X.columns:
            X[name] = SparseZeroes.copy()

In [17]:
#Now check shapes of dummy frames again
for X in OnTimeDummies: print X.shape

(98231, 717)
(98163, 717)
(98252, 717)
(98174, 717)
(98197, 717)
(98184, 717)
(98200, 717)
(98205, 717)
(98198, 717)
(98190, 717)
(98234, 717)
(98315, 717)
(98177, 717)
(98145, 717)
(88801, 717)


In [18]:
#create list of data frames with interactions included
#airport threshold: 1.5% of flights
OnTimeDummies_Inter = [interactions_aa(X, .015) for X in OnTimeDummies]

In [19]:
for X in OnTimeDummies_Inter: print X.shape

(98231, 1397)
(98163, 1397)
(98252, 1397)
(98174, 1397)
(98197, 1397)
(98184, 1414)
(98200, 1414)
(98205, 1397)
(98198, 1397)
(98190, 1414)
(98234, 1397)
(98315, 1397)
(98177, 1397)
(98145, 1397)
(88801, 1414)


In [20]:
#Once again the number of dummies is not precisely the same between batches.  We need to do the DummySet procedure again.
InterList = []
for X in OnTimeDummies_Inter:
    for name in X.columns:
        InterList.append(name)
InterSet = set(InterList)
len(InterSet)

1431

In [33]:
#Add interactions to all frames
for X in OnTimeDummies_Inter:
    SparseZeroes = pd.SparseSeries([0 for n in range(len(X))])
    for name in InterSet:
        if name not in X.columns:
            X[name] = SparseZeroes.copy()

D_DFW
C_Continental Air Lines Inc.*D_BOS
O_ERI
D_ORD
D_ORF
O_ART
C_US Airways Inc.*D_DFW
C_Southwest Airlines Co.*D_BWI
O_WYS
D_ORH
D_BFL
C_Delta Air Lines Inc.*O_MDW
C_Hawaiian Airlines Inc.*O_SFO
C_Delta Air Lines Inc.*D_MSP
C_Virgin America*D_LGA
D_DRO
O_SAN
D_ABY
C_Endeavor Air Inc.*D_SLC
C_Frontier Airlines Inc.*D_SLC
C_Frontier Airlines Inc.*O_DTW
O_XNA
O_SAF
D_ABQ
O_ANC
C_Mesa Airlines Inc.*O_SFO
D_ABI
C_SkyWest Airlines Inc.*O_MSP
O_SAV
O_SAT
C_Delta Air Lines Inc.*O_JFK
C_US Airways Inc.*O_DEN
C_Mesa Airlines Inc.*D_ORD
C_Southwest Airlines Co.*O_DEN
C_Mesa Airlines Inc.*O_BWI
C_Continental Air Lines Inc.*D_MSP
C_SkyWest Airlines Inc.*O_SLC
C_Delta Air Lines Inc.*O_SFO
O_ALO
D_HSV
O_HPN
C_SkyWest Airlines Inc.*D_LAS
C_Envoy Air*D_ATL
C_SkyWest Airlines Inc.*D_LAX
D_RSW
D_RST
O_DBQ
C_Mesa Airlines Inc.*D_SEA
O_GPT
D_DTW
D_ILG
C_US Airways Inc.*D_IAH
C_Delta Air Lines Inc.*D_SEA
C_Virgin America*D_PHX
C_Delta Air Lines Inc.*D_LGA
C_Mesa Airlines Inc.*D_ATL
C_Frontier Airlines In

In [22]:
#Now check shapes again
for X in OnTimeDummies_Inter: print X.shape

(98231, 1431)
(98163, 1431)
(98252, 1431)
(98174, 1431)
(98197, 1431)
(98184, 1431)
(98200, 1431)
(98205, 1431)
(98198, 1431)
(98190, 1431)
(98234, 1431)
(98315, 1431)
(98177, 1431)
(98145, 1431)
(88801, 1431)


In [23]:
#Divide each interaction data set into training and test data sets
from sklearn.model_selection import train_test_split

In [24]:
OnTime_XTrainFrames, OnTime_XTestFrames, OnTime_YTrainFrames, OnTime_YTestFrames = [], [], [], []
for D in OnTimeDummies_Inter:
    X_train, X_test, Y_train, Y_test = train_test_split(D.drop("ArrDelay", axis=1), D.ArrDelay)
    OnTime_XTrainFrames.append(X_train)
    OnTime_XTestFrames.append(X_test)
    OnTime_YTrainFrames.append(Y_train)
    OnTime_YTestFrames.append(Y_test)

In [25]:
#import modeling formulas
from sklearn.linear_model import SGDRegressor, SGDClassifier

In [26]:
"""1. linear regression model using SGDRegressor, penalty = L1"""
linear = SGDRegressor(penalty='l1')

In [72]:
#Use SGDRegressor partial_fit function to train model on training data
for n in range(len(OnTime_XTrainFrames)):
    print n
    #linear.partial_fit(OnTime_XTrainFrames[n], OnTime_YTrainFrames[n])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [61]:
#Check if there are any NaNs in any of the frames
ProblemDummies = []
for df in OnTimeDummies:
    ProblemDummies += df.columns[pd.isnull(df).any()].tolist()
print set(ProblemDummies)

set(['D_ITH', 'D_MVY', 'O_PPG', 'O_ITH', 'D_ADK', 'O_HYA', 'O_CNY', 'D_AZA', 'D_RFD', 'O_AZA', 'O_UST', 'O_AKN', 'D_AKN', 'O_CYS', 'D_CIU', 'O_ILG', 'O_IAG', 'D_GST', 'O_DLG', 'O_GST', 'O_PBG', 'D_DLG', 'D_STC', 'O_RFD', 'D_PPG', 'O_VEL', 'D_HYS', 'O_MVY', 'O_LWB', 'O_ADK', 'D_PUB', 'O_PUB', 'D_DVL', 'D_FOE', 'O_STC', 'O_MMH', 'D_HYA', 'D_LWB', 'D_SHD', 'O_SCE', 'D_PBG', 'D_JMS', 'O_DRT', 'D_ILG', 'D_IAG', 'D_CYS', 'D_DRT', 'O_SHD', 'D_CNY', 'O_FOE', 'D_UST'])


In [73]:
#Find problem interaction dummies
ProblemInteractions = []
for df in OnTimeDummies_Inter:
    ProblemInteractions += df.columns[pd.isnull(df).any()].tolist()
print set(ProblemInteractions) - set(ProblemDummies)

set(['C_Southwest Airlines Co.*O_MDW', 'C_JetBlue Airways*D_MDW', 'C_SkyWest Airlines Inc.*O_MDW', 'C_Hawaiian Airlines Inc.*O_MDW', 'C_Frontier Airlines Inc.*O_MDW', 'C_Alaska Airlines Inc.*O_MDW', 'C_Continental Air Lines Inc.*O_MDW', 'C_SkyWest Airlines Inc.*D_MDW', 'C_Alaska Airlines Inc.*D_MDW', 'C_Frontier Airlines Inc.*D_MDW', 'C_US Airways Inc.*D_MDW', 'C_Continental Air Lines Inc.*D_MDW', 'C_Virgin America*D_MDW', 'C_Spirit Air Lines*D_MDW', 'C_JetBlue Airways*O_MDW', 'C_United Air Lines Inc.*O_MDW', 'C_Mesa Airlines Inc.*O_MDW', 'C_Southwest Airlines Co.*D_MDW', 'C_Delta Air Lines Inc.*O_MDW', 'C_United Air Lines Inc.*D_MDW', 'C_Delta Air Lines Inc.*D_MDW', 'C_American Airlines Inc.*O_MDW', 'C_Hawaiian Airlines Inc.*D_MDW', 'C_Endeavor Air Inc.*O_MDW', 'C_ExpressJet Airlines Inc.*O_MDW', 'C_Endeavor Air Inc.*D_MDW', 'C_Envoy Air*O_MDW', 'C_Mesa Airlines Inc.*D_MDW', 'C_ExpressJet Airlines Inc.*D_MDW', 'C_Spirit Air Lines*O_MDW', 'C_American Airlines Inc.*D_MDW', 'C_Envoy Air*

In [65]:
#Why are all the airlines multiplied by Midway Airport returning NaNs?
Debug = OnTimeDummies_Inter[0]

In [66]:
Debug['C_Delta Air Lines Inc.*O_MDW']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
99971     NaN
99972     NaN
99973     NaN
99974     NaN
99975     NaN
99976     NaN
99977     NaN
99978     NaN
99979     NaN
99980     NaN
99981     NaN
99982     NaN
99983     NaN
99984     NaN
99985     NaN
99986     NaN
99987     NaN
99988     NaN
99989     NaN
99990     NaN
99991     NaN
99992     NaN
99993     NaN
99994     NaN
99995     NaN
99996     NaN
99997     NaN
99998     NaN
99999     NaN
100000    NaN
Name: C_Delta Air Lines Inc.*O_MDW, dtype: float64
BlockIndex
Block locations: array([96489])
Block lengths: array([1742])

In [69]:
Debug['C_Delta Air Lines Inc.'][100000]

0

In [70]:
Debug['O_MDW'][100000]

0

In [71]:
Debug['C_Delta Air Lines Inc.'][100000] * Debug['O_MDW'][100000]

0